In [1]:
import json
from pathlib import Path
import os

base_path = "/Users/gwonsmpro/Downloads/pillipse/class_merged_data_copy"
folder_list  = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d)) and d != '.DS_Store']


for folder in folder_list:
    folder_path = base_path + "/" + folder
    json_list = Path(folder_path).glob("*.json")
    # print(f"{folder} = {len(list(json_list))}")
    
    # JSON 파일 로드
    for json_file in json_list:
        with open(f'{str(json_file)}', 'r') as f:
            data = json.load(f)

        for shape in data['shapes']:
            if shape['label'] == 'td':
                shape['label'] = 'hb'
            elif shape['label'] == 'mn' or shape['label'] == 'hn' or shape['label'] == 'gn':
                shape['label'] = 'sn'
            elif shape['label'] == 'db' or shape['label'] == 'tb' or shape['label'] == 'zh' or shape['label'] == 'sc2' or shape['label'] == 'sc3':
                shape['label'] = 'sc'
            elif shape['label'] == 'embo' or shape['label'] == 'embox':
                shape['label'] = 'ab'
            elif shape['label'] == "test":
                shape['label'] = 'ul'

        # 결과를 새 JSON 파일에 저장
        with open(f'{str(json_file)}', 'w') as f:
            json.dump(data, f, indent=4)




In [2]:
import os
import shutil

# 정의된 경로
base_path = "/Users/gwonsmpro/Downloads/pillipse/class_merged_data_test_merged"
dest_path_image = "/Users/gwonsmpro/Downloads/pillipse/path_to_image"
dest_path_label = "/Users/gwonsmpro/Downloads/pillipse/path_to_label"

# 이미지와 라벨을 저장할 경로가 존재하지 않으면 생성합니다.
os.makedirs(dest_path_image, exist_ok=True)
os.makedirs(dest_path_label, exist_ok=True)

# base_path에서 모든 디렉토리를 순회하면서 파일을 복사합니다.
for dirpath, dirnames, filenames in os.walk(base_path):
    for filename in filenames:
        if filename.endswith('.png'):
            # 이미지 파일은 dest_path_image에 복사합니다.
            shutil.copy2(os.path.join(dirpath, filename), dest_path_image)
        elif filename.endswith('.json'):
            # 텍스트 파일은 dest_path_label에 복사합니다.
            shutil.copy2(os.path.join(dirpath, filename), dest_path_label)
